##Задача: Восстановление пропущенных пробелов в тексте с помощью NLP / DL / алгоритма.
#**Как решать?**



1.   Классический метод NLP (n-gram language model)
2.   DL метод NLP (fine-tuning BERT)

В данном случае нам нужно найти точное, быстрое и легковесное решение. Поэтому я решил проверить две гипотезы:


*   Решить задачу легковесным и простым способом, при котором нам не нужно много вычислительных мощностей (будем делать все на CPU), но при этом потеряем качество решения.
*   Решить задачу более сложным способом, при котором нам нужно достаточно вычислительных мощностей, но при этом мы выиграем в качестве решения задачи.

**Суть:**
Проверить, что будет лучше и логичнее для бизнеса. Возможно легковесное решение будет хорошо справляться с задачей и нам не нужно дообучать BERT, а может быть и наоборот, что мы потеряем эффективность и точность, из-за чего классический метод тут совсем не подойдет.

Обучение n-gram language model производилось на CPU в Google Colab

Обучение BERT производилось на T4 GPU в Google Colab




In [1]:
import re
import os
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm.auto import tqdm
import kagglehub
import math
from datasets import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
import torch
%matplotlib inline

#N-gram model
В данном случае мы будем реализовать модель, которая не будет генерировать последовательность токенов, а наоборот проверять, насколько вероятно, что то или иное разбиение строки на символы, является именно тем словом. А разбивать будет именно алогритм. То есть получаем решение: динамическое программирование + словарь + вероятностная модель (n-граммы, unigram). Для этого нам необходимо найти специфичный датасет, который поможет нам решить задачу. Поискав в интернете, я нашел датасет на каггле avito-dataset. Он идеально для нас подходит, потому что у нас есть названия разных объявлений и их описаний. Это дает нам преимущество в том, что сохраняется специфичность запросов.
Подгружаем его

In [3]:
path = kagglehub.dataset_download("vitaliy3000/avito-dataset")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'avito-dataset' dataset.
Path to dataset files: /kaggle/input/avito-dataset


In [4]:
data = pd.read_csv('/kaggle/input/avito-dataset/train.csv') # если не запускается ячейка, то запустите прошлую ячейку еще раз

In [ ]:
data

Нам нужен список из названий объявлений и их описаний, поэтому создаем его

In [ ]:
lines = data.apply(lambda row: row['title'] + '. ' + row['description'], axis=1).tolist()
lines

Очищаем текст от служебных знаков, лишних смайликов и тд

In [6]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s\.\,\!\?\-\:]', ' ', text)
    return text.strip()

In [ ]:
clean_lines = [clean_text(line) for line in tqdm(lines)]
clean_lines

И присутпаем к созданию n-gram модели. Определим функцию, которая считает n-gramы

In [8]:
# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Подсчитывает, сколько раз каждое слово встречалось после (n - 1) предыдущих слов.

    :param lines: итерируемый объект со строками пробельно-разделенных токенов
    :returns: словарь { кортеж(токены_префикса): {следующий_токен_1: счетчик_1, следующий_токен_2: счетчик_2}}
    """

    counts = defaultdict(Counter)
    for line in tqdm(lines):
        tokens = line.split() + [EOS]

        for i in range(len(tokens)):
            prefix = []

            for j in range(n - 1):
                pos = i - (n - 1) + j
                if pos < 0:
                    prefix.append(UNK)
                else:
                    prefix.append(tokens[pos])
            prefix = tuple(prefix)

            next_token = tokens[i]

            counts[prefix][next_token] += 1

    return counts

Определим языковую n граммную модель со сглаживанием Лапласа. В данном случае нам крайне необходимо определить не классическую n граммную модель, а именно со сглаживанием, так как иначе незнакомые слова будут получать нулевые вероятности, что чревато потерей информации и неустойчивость модели к словам, отсутствующим в тренировочном корпусе

In [9]:
class LaplaceLanguageModel:
    def __init__(self, lines, n, delta=1.0, len_decay=0.3):
        self.n = n
        self.delta = delta
        self.len_decay = len_decay

        # считаем n-граммы и униграммы
        self.uni = Counter()
        for line in lines:
            self.uni.update(line.strip().split())

        counts = count_ngrams(lines, self.n)
        self.vocab = set(t for d in counts.values() for t in d)
        self.V = max(1, len(self.vocab))
        self.total_uni = sum(self.uni.values()) + delta * (self.V + 1)

        self.probs = defaultdict(dict)
        for prefix, token_counts in counts.items():
            total = sum(token_counts.values()) + delta * self.V
            self.probs[prefix] = {t: (c + delta) / total for t, c in token_counts.items()}

    def _norm_prefix(self, prefix):
        p = prefix.split()[-(self.n-1):]
        return tuple([UNK] * (self.n - 1 - len(p)) + p)

    def get_possible_next_tokens(self, prefix):
        return self.probs[self._norm_prefix(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        seen = self.get_possible_next_tokens(prefix)
        if next_token in seen:
            return seen[next_token]

        # backoff к униграммам, если слово вообще видели
        if self.uni[next_token] > 0:
            return (self.uni[next_token] + self.delta) / self.total_uni

        # совсем неизвестное слово: сильный штраф по длине
        L = max(1, len(next_token))
        return (1.0 / (self.V + 1)) * (self.len_decay ** (L - 1))


In [10]:
lm = LaplaceLanguageModel(lines, n=3, delta=0.5)

  0%|          | 0/489517 [00:00<?, ?it/s]

Проверяем работоспосбность модели

In [11]:
possible_tokens = lm.get_possible_next_tokens('купить айфон с')

tokens, probs = zip(*possible_tokens.items())
print(f'Tokens: {tokens} - probs: {probs}')
random.choices(tokens, k=1)[0]

Tokens: ('моей', 'вашей', 'доплатой', 'разбитым', 'небольшой', 'документами,', 'новой', 'надписью', 'рабочем', 'доставкой', 'августа', 'док.', 'доплатой!', 'доплатой,', 'документами,не', 'доплатой.Комплект,коробка,документы,зарядное', 'небес') - probs: (1.9006921140939598e-05, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06, 1.9662332214765103e-06)


'надписью'

In [12]:
BOS = "<s>"
UNK = "<unk>"

def restore_spaces(text, language_model):
    """
    Восстанавливает пробелы в тексте с помощью Viterbi и n-грамм LM.
    language_model:
        .n  -> порядок n-грамм
        .get_next_token_prob(context_str, next_word_str) -> float in (0,1]
    """
    n = language_model.n
    L = len(text)
    max_tok_len = 20

    dp = [(-float("inf"), -1) for _ in range(L + 1)]
    dp[0] = (0.0, -1)

    for i in range(1, L + 1):
        j_start = max(0, i - max_tok_len)
        for j in range(j_start, i):
            prev_prob, _ = dp[j]
            if prev_prob == -float("inf"):
                continue

            word = text[j:i]

            if j == 0:
                context_words = [BOS] * (n - 1)
            else:
                context_words = []
                pos = j
                for _ in range(n - 1):
                    if pos <= 0:
                        context_words.append(BOS)
                    else:
                        prev_pos = dp[pos][1]
                        if prev_pos < 0:
                            context_words.append(BOS)
                            pos = 0
                        else:
                            context_words.append(text[prev_pos:pos])
                            pos = prev_pos
                context_words.reverse()

            context = " ".join(context_words)

            p = language_model.get_next_token_prob(context, word)
            if p <= 0.0:
                continue

            total = prev_prob + math.log(p)
            if total > dp[i][0]:
                dp[i] = (total, j)

    if dp[L][0] == -float("inf"):
        return text

    parts = []
    pos = L
    while pos > 0:
        _, prev_pos = dp[pos]
        if prev_pos < 0:
            parts.append(text[0:pos])
            break
        parts.append(text[prev_pos:pos])
        pos = prev_pos

    return " ".join(reversed(parts)).strip()


In [13]:
print(restore_spaces('работавМосквеудаленно', lm)) # проверяем работу алгоритма

работа в Москве удаленно


In [ ]:
# Читаем файл как один столбец
df = pd.read_csv('dataset_1937770_3.txt', header=None, names=['data'], sep='SEP')

# Разделяем данные на два столбца по первой запятой
df[['part1', 'part2']] = df['data'].str.split(',', n=1, expand=True)

# Удаляем исходный столбец
df.drop(columns=['data'], inplace=True)

# Из первой строки делаем header
new_header = df.iloc[0]
df = df[1:].set_axis(new_header, axis=1)
display(df.head())

In [ ]:
lines_no_spaces = df.apply(lambda row: row['text_no_spaces'], axis=1).tolist()
lines_no_spaces

In [ ]:
lines_with_spaces = [restore_spaces(line, lm) for line in lines_no_spaces]
lines_with_spaces # запускаем алгоритм

In [17]:
def find_space_positions(original, corrected):
    """
    Находит позиции, куда нужно добавить пробелы в оригинальной строке,
    чтобы получить исправленную строку.
    :param original: Исходная строка без пробелов
    :param corrected: Исправленная строка с пробелами
    :returns: Список позиций (индексов) для добавления пробелов
    """
    i, j = 0, 0
    space_positions = []

    while i < len(original) and j < len(corrected):
        if original[i] == corrected[j]:
            i += 1
            j += 1
        elif corrected[j] == ' ':
            space_positions.append(i)
            j += 1
        else:
            return None

    while j < len(corrected) and corrected[j] == ' ':
        space_positions.append(i)
        j += 1

    return space_positions


In [18]:
find_space_positions('куплюайфон14про','куплю айфон 14 про') # проверяем работоспособность

[5, 10, 12]

В данном случае мы обучаем языковую модель на title + description, а значит n граммы не всегда похожи на запросы авито. Например, обычно пишут: "куплю айфон 14", "ищу друга для общения", "сдам квартиру" и так далее. У нас в датасете больше "айфон 5", "картина" и тд. Поэтому можно взять список разных ключевых слов, которые очень часто повторяются в запросах людей. А также просто привести строку к формату, где все будет по правилам русского языка: поставим пробел после знаков препинания (если нет пробела), перед открывающими скобками и после закрывающих, после чисел перед словами и так далее. Это должно сильно повысить f1 меру.

In [ ]:
def add_missing_spaces(text):
    """
    Добавляет пропущенные пробелы в текст в соответсвии с пунктуацией, сменой языков и регистром
    """
    if pd.isna(text):
        return text

    # Основные правила для добавления пробелов
    patterns = [
        # После знаков препинания (если нет пробела)
        (r'([.,!?;:])([а-яА-Яa-zA-Z])', r'\1 \2'),
        # Перед открывающими скобками и после закрывающих
        (r'([а-яА-Яa-zA-Z])(\()', r'\1 \2'),
        (r'(\))([а-яА-Яa-zA-Z])', r'\1 \2'),
        # Вокруг тире (если оно используется как знак препинания)
        (r'([а-яА-Яa-zA-Z])—([а-яА-Яa-zA-Z])', r'\1 — \2'),
        # После чисел перед словами
        (r'(\d)([а-яА-Яa-zA-Z])', r'\1 \2'),
        # Перед числами после слов
        (r'([а-яА-Яa-zA-Z])(\d)', r'\1 \2'),
        # Перед заглавными буквами
        (r'([а-яa-z])([А-ЯA-Z])', r'\1 \2'),
        # При переходе с русского на английскйй
        (r'([а-яА-Я])([A-Za-z])', r'\1 \2'),
        # При переходе с английского на русский
        (r'([A-Za-z])([а-яА-Я])', r'\1 \2'),
    ]

    result = str(text)

    # Применяем все правила
    for pattern, replacement in patterns:
        result = re.sub(pattern, replacement, result)

    # Убираем лишние пробелы
    result = re.sub(r'\s+', ' ', result).strip()

    return result

def add_spaces_by_keywords(text):
    """
    Добавляет пропущенные пробелы в текст в соответсвии с наиболее логичными первыми словами в запросе

    Тренировочный датасет содержал только тексты объявлений, поэтому слова, обозначающие намерение купить товар, проверяются отдельным словарем

    """

    # Слова, обозначающие намерение купить товар
    patterns = ["ищу", "куплю", "нужно", "хочу"] #

    # Добавление пробела после слов
    pattern = '(' + '|'.join(patterns) + ')'
    result = re.sub(pattern, r'\1 ', text, flags=re.IGNORECASE)

    # Убираем лишние пробелы
    result = re.sub(r'\s+', ' ', result).strip()

    return result

spaces_in_lines = [add_spaces_by_keywords(add_missing_spaces(line)) for line in lines_with_spaces]
spaces_in_lines

In [ ]:
# составляем список из списков позиций пробела в строке
# готовим файл к сдачи
spaces_in_lines = [str(find_space_positions(lines_no_spaces[i], spaces_in_lines[i])) for i in range(len(lines_with_spaces))]
spaces_in_lines

In [21]:
# создаем df и сохраняем файл
submissions = pd.DataFrame({
    'id': range(len(spaces_in_lines)),
    'predicted_positions': spaces_in_lines
})

submissions.to_csv('submissions.csv', index=True, encoding='utf-8')

In [ ]:
submissions

,id,predicted_positions
0,0,[]
1,1,[7]
2,2,"[20, 21]"
3,3,"[5, 10, 18]"
4,4,[]
...,...,...
1000,1000,[]
1001,1001,[]
1002,1002,"[3, 5]"
1003,1003,"[19, 22]"


In [ ]:
type(submissions['predicted_positions'][0])

Итог:

*   LaplaceLM(n=3, delta=0.5) - F1 = 37%
*   LaplaceLM(n=3, delta=0.5) + эвристика - F1 = 54% (примерно, как результат на степике)

Первый реузльтат взят со степика, второй проверен на отдельном искусственном датасете (F1 на реальном датасете меньше на 1-2%, чем F1 на моем датасете).



##Fine-tuning BERT
Задача восстановления пропущенных пробелов в тексте может быть сведена к задаче бинарной классификации на уровне символов: для каждой позиции между символами предсказывается наличие (1) или отсутствие (0) пробела.

BERT идеально подходит для этой задачи, поскольку его архитектура позволяет эффективно анализировать контекстные зависимости между символами, а предобучение на больших текстовых корпусах обеспечивает хорошие начальные веса.

# Данные
Для обучения нам нужны данные. Воспользуемся уже знакомы датасетом avito-dataset. Из него я предлагаю взять только названия объявлений, так как они больше всего похожи на запросы пользователей, например, "куплю айфон 14 про". Чтобы дообучить BERT, нам надо исскуственно подогнать датасет под нас. Сделаем две колонки:

*   Признак - текст без пробелов
*   Таргет - текст с пробелами



In [ ]:
titles = data.apply(lambda row: row['title'], axis=1).tolist()
titles

In [ ]:
titles_no_space = [title.replace(' ', '') for title in titles]
titles_no_space

In [ ]:
dataFrame = pd.DataFrame({
    'input': titles_no_space,
    'target': titles,
})
dataFrame

Теперь, когда у нас есть датасет, подготовим данные. Нам нужно воспользоваться функцией find_space_positions, которая вернет нам списки индексов, где должны стоять пробелы

In [ ]:
def prepare_dataset(df):
    """Подготовка датасета для BERT"""
    samples = []

    for idx, row in df.iterrows():
        text_with_spaces = row['target']  # строка с пробелами
        text_without_spaces = row['input']  # строка без пробелов

        text_with_spaces = re.sub(r'\s+', ' ', text_with_spaces).strip()

        space_positions = find_space_positions(text_without_spaces, text_with_spaces)

        samples.append({
            'text': text_without_spaces,
            'labels': space_positions,
        })

    return pd.DataFrame(samples)
prepare_dataset = prepare_dataset(dataFrame)
print(prepare_dataset)

In [38]:
train, test = train_test_split(prepare_dataset, test_size=0.1, random_state=42, shuffle=True) # разделим данные на train и test

In [ ]:
train

Определим класс SpaceDataset, чтобы  реализовать character-level classification, где:

Каждый символ → отдельный токен

Для каждого символа предсказываем: пробел после него или нет

In [14]:
class SpaceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        true_labels = self.labels[idx]

        # Токенизация по символам
        tokens = list(text)
        inputs = self.tokenizer(
            tokens,
            is_split_into_words=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Создание масок и меток
        label_ids = torch.zeros(self.max_length, dtype=torch.long)
        attention_mask = torch.zeros(self.max_length, dtype=torch.long)

        for i in range(min(len(tokens), self.max_length)):
            attention_mask[i] = 1
            if i in true_labels:
                label_ids[i] = 1  # SPACE
            else:
                label_ids[i] = 0  # NO_SPACE

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': attention_mask,
            'labels': label_ids
        }


#Цикл обучения
Здесь мы определяем модель. Использовать будем "cointegrated/rubert-tiny", потому что она легкая и русскоязычная, как раз под нашу задачу.

Обучение проводил два раза, на 3х эпохах и на 6, чтобы посмотреть на качество модели. Хотелось понять, будет ли качество лучше.

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"

device = ('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'cointegrated/rubert-tiny' # легкая русскоязычная модель
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NO_SPACE", 1: "SPACE"},
    label2id={"NO_SPACE": 0, "SPACE": 1}
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Датасеты
train_dataset = SpaceDataset(train['text'].tolist(),
                            train['labels'].tolist(), tokenizer)
val_dataset = SpaceDataset(test['text'].tolist(),
                          test['labels'].tolist(), tokenizer)

# Аргументы обучения
training_args = TrainingArguments(
    output_dir='./results',
    report_to=None,
    logging_dir='./logs',
    logging_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

In [60]:
def predict_spaces(text_without_spaces, model, tokenizer, threshold=0.7):
    """Предсказание пробелов для нового текста"""
    model.eval()

    tokens = list(text_without_spaces)
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors='pt',
        max_length=128,
        padding=True,
        truncation=True
    )

    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=-1)

    space_positions = []
    for i in range(min(len(tokens), probabilities.shape[1])):
        if probabilities[0, i, 1] > threshold:  # уверенность в пробеле
            space_positions.append(i)

    result = list(text_without_spaces)
    for pos in sorted(space_positions, reverse=True):
        if pos < len(result):
            result.insert(pos, ' ')

    return ''.join(result), space_positions

def quick_test(model, tokenizer):
    ""
    test_cases = [
        'куплюайфон14про',
        'книгавхорошемсостоянии',
        'стульяизпрессованнойкожи'
    ]

    for test_text in test_cases:
        result, positions = predict_spaces(test_text, model, tokenizer)
        print(f"Вход:  {test_text}")
        print(f"Выход: {result}")
        print(f"Позиции: {positions}")
        print("-" * 50)

quick_test(model, tokenizer)

Вход:  куплюайфон14про
Выход: куплю айфон 14 про
Позиции: [5, 10, 12]
--------------------------------------------------
Вход:  книгавхорошемсостоянии
Выход: книга в хорошем состоянии
Позиции: [5, 6, 13]
--------------------------------------------------
Вход:  стульяизпрессованнойкожи
Выход: стулья из прессованной кожи
Позиции: [6, 8, 20]
--------------------------------------------------


In [ ]:
df = pd.read_csv('dataset_1937770_3.txt', header=None, names=['data'], sep='SEP')
df[['part1', 'part2']] = df['data'].str.split(',', n=1, expand=True)
df.drop(columns=['data'], inplace=True)
new_header = df.iloc[0]
df = df[1:].set_axis(new_header, axis=1)
display(df.head())

In [ ]:
lines_no_spaces = df.apply(lambda row: row['text_no_spaces'], axis=1).tolist()
lines_no_spaces

In [ ]:
spaces_in_lines = [predict_spaces(line, model, tokenizer)[0] for line in lines_no_spaces]
spaces_in_lines # расставляем пробелы через BERT

Здесь, также как и в случае в LaplaceLM, решил попробовать улучшить f1 через эвристику. Добавил эти функции снова, чтобы не возвращаться в другие ячейки

In [ ]:
def add_missing_spaces(text):
    """
    Добавляет пропущенные пробелы в текст в соответсвии с пунктуацией, сменой языков и регистром
    """
    if pd.isna(text):
        return text

    # Основные правила для добавления пробелов
    patterns = [
        # После знаков препинания (если нет пробела)
        (r'([.,!?;:])([а-яА-Яa-zA-Z])', r'\1 \2'),
        # Перед открывающими скобками и после закрывающих
        (r'([а-яА-Яa-zA-Z])(\()', r'\1 \2'),
        (r'(\))([а-яА-Яa-zA-Z])', r'\1 \2'),
        # Вокруг тире (если оно используется как знак препинания)
        (r'([а-яА-Яa-zA-Z])—([а-яА-Яa-zA-Z])', r'\1 — \2'),
        # После чисел перед словами
        (r'(\d)([а-яА-Яa-zA-Z])', r'\1 \2'),
        # Перед числами после слов
        (r'([а-яА-Яa-zA-Z])(\d)', r'\1 \2'),
        # Перед заглавными буквами
        (r'([а-яa-z])([А-ЯA-Z])', r'\1 \2'),
        # При переходе с русского на английскйй
        (r'([а-яА-Я])([A-Za-z])', r'\1 \2'),
        # При переходе с английского на русский
        (r'([A-Za-z])([а-яА-Я])', r'\1 \2'),
    ]

    result = str(text)

    # Применяем все правила
    for pattern, replacement in patterns:
        result = re.sub(pattern, replacement, result)

    # Убираем лишние пробелы
    result = re.sub(r'\s+', ' ', result).strip()

    return result

def add_spaces_by_keywords(text):
    """
    Добавляет пропущенные пробелы в текст в соответсвии с наиболее логичными первыми словами в запросе

    Тренировочный датасет содержал только тексты объявлений, поэтому слова, обозначающие намерение купить товар, проверяются отдельным словарем

    """

    # Слова, обозначающие намерение купить товар
    patterns = ["ищу", "куплю", "нужно", "хочу"] #

    # Добавление пробела после слов
    pattern = '(' + '|'.join(patterns) + ')'
    result = re.sub(pattern, r'\1 ', text, flags=re.IGNORECASE)

    # Убираем лишние пробелы
    result = re.sub(r'\s+', ' ', result).strip()

    return result

spaces_in_lines = [add_spaces_by_keywords(add_missing_spaces(line)) for line in spaces_in_lines]
spaces_in_lines # получили список обработанных строк

In [ ]:
result = [str(find_space_positions(line.replace(' ', ''), line)) for line in spaces_in_lines]
result # получаем позиции пробелов в строках

In [70]:
# создаем файл и сохраняем результаты
submissions = pd.DataFrame({
    'id': range(len(spaces_in_lines)),
    'predicted_positions': result
})

submissions.to_csv('submissions.csv', index=True, encoding='utf-8')

Итог:

*   3 эпохи - F1 = 71%
*   6 эпох - F1 = 76%
*   BERT (6 эпох) + эвристика - F1 = 79%

Результаты брал с степика.





## Вывод
Если сравнивать два способа, то очевидно, что BERT по качеству побеждает, причем сильно (79 - 34 = 45% разница). Но стоит понимать, что BERT использует сильно больше вычислительных ресурсов, из-за чего не всегда оно того стоит. Тем более, что эвристика + LaplaceLM уже сокращает разницу (79 - 54 = 25%).

Поэтому тут появляется выбор:


*   Потерять в качестве, но выиграть в ресурсах - LaplaceLM
*   Потерять в ресурсах, но выиграть в качестве - BERT

Также стоит отметить, что в датасете были не только объявления с авито, а еще и текста разных песен, из-за чего мои модели выдавали качество меньше, чем могло быть на реальных кейсах с авито. Это произошло из-за того, что я обучал их на специфичном датасете, связанном конкретно с Авито.
Без строчек песен я получил результаты:


*   LaplaceLM - F1 = 77%
*   BERT - F1 = 92%

